In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

In [2]:

sim = AerSimulator()

In [3]:

# Bell state preparation functions
def bell_state(state_id):
    """
    0 -> |Φ+> = (|00> + |11>)/√2
    1 -> |Φ-> = (|00> - |11>)/√2
    2 -> |Ψ+> = (|01> + |10>)/√2
    3 -> |Ψ-> = (|01> - |10>)/√2
    """
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)

    if state_id == 1:   # |Φ->
        qc.z(0)
    elif state_id == 2: # |Ψ+>
        qc.x(0)
    elif state_id == 3: # |Ψ->
        qc.x(0)
        qc.z(0)

    return qc

In [4]:

# Superdense coding test for one Bell state
def superdense_test(state_id, c, d):
    """
    c -> Alice applies X if c=1
    d -> Alice applies Z if d=1
    Bob performs Bell measurement and decodes
    """
    # Registers
    A = QuantumRegister(1, "Alice")
    B = QuantumRegister(1, "Bob")
    Cc = ClassicalRegister(1, "c_out")
    Cd = ClassicalRegister(1, "d_out")

    qc = QuantumCircuit(A, B, Cc, Cd)

    # --- Prepare bell state
    bell = bell_state(state_id)
    qc.compose(bell, [A[0], B[0]], inplace=True)
    qc.barrier()

    # --- Alice encodes classical bits (c,d)
    if d == 1:
        qc.z(A[0])
    if c == 1:
        qc.x(A[0])
    qc.barrier()

    # --- Bob's decoding
    qc.cx(A[0], B[0])
    qc.h(A[0])
    qc.barrier()

    # --- Bob measures the two qubits
    qc.measure(A[0], Cc)
    qc.measure(B[0], Cd)

    # run simulator
    result = sim.run(qc).result()
    counts = result.get_counts()

    return counts, qc



In [5]:

# Test all Bell states with all 4 possible messages
bell_names = ["Φ+", "Φ−", "Ψ+", "Ψ−"]
messages = [(0,0), (0,1), (1,0), (1,1)]

for state_id in range(4):
    print(f"\n=== Testing Bell State: |{bell_names[state_id]}> ===")
    for c, d in messages:
        counts, circuit = superdense_test(state_id, c, d)
        print(f"Alice sent (c={c}, d={d}) → Bob received {list(counts.keys())}")



=== Testing Bell State: |Φ+> ===
Alice sent (c=0, d=0) → Bob received ['0 0']
Alice sent (c=0, d=1) → Bob received ['0 1']
Alice sent (c=1, d=0) → Bob received ['1 0']
Alice sent (c=1, d=1) → Bob received ['1 1']

=== Testing Bell State: |Φ−> ===
Alice sent (c=0, d=0) → Bob received ['0 1']
Alice sent (c=0, d=1) → Bob received ['0 0']
Alice sent (c=1, d=0) → Bob received ['1 1']
Alice sent (c=1, d=1) → Bob received ['1 0']

=== Testing Bell State: |Ψ+> ===
Alice sent (c=0, d=0) → Bob received ['1 0']
Alice sent (c=0, d=1) → Bob received ['1 1']
Alice sent (c=1, d=0) → Bob received ['0 0']
Alice sent (c=1, d=1) → Bob received ['0 1']

=== Testing Bell State: |Ψ−> ===
Alice sent (c=0, d=0) → Bob received ['1 1']
Alice sent (c=0, d=1) → Bob received ['1 0']
Alice sent (c=1, d=0) → Bob received ['0 1']
Alice sent (c=1, d=1) → Bob received ['0 0']
